In [40]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from torchsummary import summary

In [1]:
from efficientnet_pytorch import EfficientNet
# model = EfficientNet.from_pretrained('efficientnet-b7')

In [44]:
# class Efficientnet(nn.Module):

#     def __init__(self, out_size):
#         super().__init__()
#         self.model = EfficientNet.from_pretrained('efficientnet-b7')
        
#         for param in self.model.parameters():
#             param.requires_grad = False
#         self.model._fc = nn.Linear(2560, 350)
        
#     def forward(self, image):
#         return self.model(image)


class Efficientnet(nn.Module):
    def __init__(self, out_size):
        super().__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b7')

        for param in self.model.parameters():
            param.requires_grad = False
        
    def forward(self, image):
        f = self.model.extract_features(image)
        conv = nn.Sequential(nn.Conv2d(2560, 350, 7))
        f = conv(f)
        return f.squeeze(-1).squeeze(-1)

In [29]:
model = Efficientnet(350)
# linear at the last
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 225, 225]               0
Conv2dStaticSamePadding-2         [-1, 64, 112, 112]           1,728
       BatchNorm2d-3         [-1, 64, 112, 112]             128
         ZeroPad2d-4         [-1, 64, 114, 114]               0
Conv2dStaticSamePadding-5         [-1, 64, 112, 112]             576
       BatchNorm2d-6         [-1, 64, 112, 112]             128
          Identity-7             [-1, 64, 1, 1]               0
Conv2dStaticSamePadding-8             [-1, 16, 1, 1]           1,040
          Identity-9             [-1, 16, 1, 1]               0
Conv2dStaticSamePadding-10             [-1, 64, 1, 1]           1,088
         Identity-11         [-1, 64, 112, 112]               0
Conv2dStaticSamePadding-12         [-1, 32, 112, 112]           2,048
      BatchNorm2d-13         [-1, 32, 112, 112]              64
      MBConv

In [45]:
model = Efficientnet(350)
#conv at the last
summary(model, (3, 224, 224))

Loaded pretrained weights for efficientnet-b7


TypeError: conv2d(): argument 'input' (position 1) must be Tensor, not int

In [36]:
class DenseNet(nn.Module):
    def __init__(self, out_size):
        super().__init__()
        model = models.densenet161(pretrained=True)

        for param in model.parameters():
            param.requires_grad = False

        model = list(model.children())[:-1]
        model.append(nn.Conv2d(2208, out_size, 7))
        self.net = nn.Sequential(*model)

    def forward(self, image):
        return self.net(image).squeeze(-1).squeeze(-1)

In [37]:
model = DenseNet(350)
# linear at the last
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 96, 112, 112]          14,112
       BatchNorm2d-2         [-1, 96, 112, 112]             192
              ReLU-3         [-1, 96, 112, 112]               0
         MaxPool2d-4           [-1, 96, 56, 56]               0
       BatchNorm2d-5           [-1, 96, 56, 56]             192
              ReLU-6           [-1, 96, 56, 56]               0
            Conv2d-7          [-1, 192, 56, 56]          18,432
       BatchNorm2d-8          [-1, 192, 56, 56]             384
              ReLU-9          [-1, 192, 56, 56]               0
           Conv2d-10           [-1, 48, 56, 56]          82,944
      BatchNorm2d-11          [-1, 144, 56, 56]             288
             ReLU-12          [-1, 144, 56, 56]               0
           Conv2d-13          [-1, 192, 56, 56]          27,648
      BatchNorm2d-14          [-1, 192,

In [61]:
ad.weight[0].type


<function Tensor.type>

In [38]:
a = torch.tensor(np.random.randn(1, 3, 224, 224)).float()

Efficientnet(350).forward(a)

Loaded pretrained weights for efficientnet-b7


tensor([[ 3.3844e-02,  9.8660e-02, -1.1486e-01, -6.6111e-01,  4.5362e-01,
         -1.2611e-01, -1.9243e-01,  1.4289e-01,  3.9581e-01,  4.7578e-01,
         -4.8986e-01,  2.4874e-01,  9.9185e-02, -3.2185e-01, -9.7896e-02,
         -2.2306e-01, -4.3514e-01, -4.1889e-01,  5.2712e-01, -5.6721e-02,
         -3.2741e-01, -1.0435e-01,  9.6384e-03,  8.5452e-02, -6.0131e-01,
          1.4884e-01,  5.7588e-01,  1.7584e-01, -8.5254e-03, -1.3952e-01,
          6.8691e-02, -1.8392e-01,  2.4088e-01, -3.2038e-02,  1.4199e-01,
          1.1852e-01, -2.2490e-01, -2.3965e-01,  1.4475e-01, -1.5525e-01,
          1.2225e-01,  3.2920e-01, -1.2241e-01,  4.4106e-01, -3.2477e-01,
         -6.3307e-01, -5.4005e-01, -5.3225e-02,  1.4055e-01, -4.4887e-01,
         -1.6870e-01, -2.4077e-01,  2.8588e-02, -1.5282e-01,  3.7146e-01,
          8.0630e-02, -6.0527e-01,  4.4160e-01, -5.1576e-01,  4.0540e-01,
         -2.9273e-01, -1.1522e-01, -5.7398e-01, -3.5874e-02, -6.1895e-03,
          8.3754e-01,  1.7464e-01,  5.

In [26]:
a

tensor([[[[-0.9267, -0.0686,  1.4157,  ..., -0.1040, -2.2969,  0.6133],
          [ 1.0711, -0.3885,  0.7886,  ..., -0.4044, -1.8349, -0.4463],
          [ 0.5750,  0.5393, -1.8781,  ..., -1.4553,  0.8965,  1.4657],
          ...,
          [-0.5953, -0.1211,  0.6487,  ..., -1.0072,  1.1473, -1.1693],
          [ 0.5543,  0.8344, -2.0776,  ..., -1.6852, -1.2548, -1.3349],
          [-1.2716, -0.0114, -0.5155,  ...,  0.6600,  0.4248, -0.5612]],

         [[-1.0177,  1.2683,  1.5925,  ...,  0.2382, -0.9284, -0.0485],
          [ 0.5013, -2.0085, -0.6077,  ...,  0.1829, -0.9771, -1.2250],
          [-1.0253,  0.7702,  0.8693,  ..., -0.3867,  1.3401,  1.6142],
          ...,
          [ 0.3129,  0.5523,  0.3327,  ..., -0.7779, -0.2204, -0.8259],
          [ 0.3035, -0.2075,  0.2033,  ...,  0.1536, -0.2167, -3.3154],
          [ 0.1382, -1.4978,  0.9989,  ...,  2.2461,  0.4700, -0.1665]],

         [[ 0.0781,  1.1125,  0.9668,  ...,  1.1830, -0.1493,  0.1973],
          [ 2.5552,  0.0629,  

In [22]:
summary(net, input_size=(3,224,224))

NotImplementedError: 

In [34]:
model = models.densenet161(pretrained=True)

# for param in model.parameters():
#     param.requires_grad = False

model = list(model.children())[:-1]
model.append(nn.Conv2d(2560, 350, 7))
net = nn.Sequential(*model)

In [7]:
# list(net.parameters())[-2]a

In [35]:
summary(net, input_size=(3,224,224))

RuntimeError: Given groups=1, weight of size 350 2560 7 7, expected input[2, 2208, 7, 7] to have 2560 channels, but got 2208 channels instead

In [ ]:
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [3]:
model = models.densenet161(pretrained=True)

In [27]:
out_size = 350
model = models.densenet161(pretrained=True)
# model = list(model.children())[:-1]
# model.append(nn.Conv2d(512, out_size, 1))
# net = nn.Sequential(*model)

In [ ]:
2208 linear

In [24]:
list(model.children())[:-1]

AttributeError: 'list' object has no attribute 'children'

In [28]:
model = list(model.children())[:-1]
model.append(nn.Conv2d(2208, out_size, 7))
net = nn.Sequential(*model)

In [29]:
summary(net, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 96, 112, 112]          14,112
       BatchNorm2d-2         [-1, 96, 112, 112]             192
              ReLU-3         [-1, 96, 112, 112]               0
         MaxPool2d-4           [-1, 96, 56, 56]               0
       BatchNorm2d-5           [-1, 96, 56, 56]             192
              ReLU-6           [-1, 96, 56, 56]               0
            Conv2d-7          [-1, 192, 56, 56]          18,432
       BatchNorm2d-8          [-1, 192, 56, 56]             384
              ReLU-9          [-1, 192, 56, 56]               0
           Conv2d-10           [-1, 48, 56, 56]          82,944
      BatchNorm2d-11          [-1, 144, 56, 56]             288
             ReLU-12          [-1, 144, 56, 56]               0
           Conv2d-13          [-1, 192, 56, 56]          27,648
      BatchNorm2d-14          [-1, 192,

In [77]:
def get_random_idx(x):
    np.random.choice(np.argwhere(x == 1).reshape(-1), 1)

In [75]:
x = np.array([0, 1.0, 1.0, 0, 0, 0, 1])
# np.random.shuffle(x)
np.random.choice(np.argwhere(x == 1).reshape(-1), 1)

array([1])

In [76]:
np.argwhere(x == 1)

array([[1],
       [2],
       [6]])

In [42]:
np.random.shuffle(np.array([0, 1, 2, 3, 4, 5, 6]))

In [40]:
def sett(**args):
    print(args)

In [41]:
sett(**(1, 2, 3, 4))

TypeError: sett() argument after ** must be a mapping, not tuple